<a href="https://colab.research.google.com/github/Rezaie/Kire_mama/blob/master/tansformer_optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Multivariate time series prediction using transformer with hyperparameter optimization

In this notebook, we use **Optuna** to find the optimum values of hyperparameters. In this notebook we specifically optimize the values of **learning rate, weight decay, positional encoding dropout** and **encoder layer dropout**.

Optuna is a python package specifially designed for hyperparameter tuning. We need to define a range of possible values for each of the hyperparameters. And optuna will try different parameter values with the model to minimize the validation loss after for specified number of experiments.


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 17.4 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import math

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import optuna

In [ ]:
path = 'final_data.csv'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Implement determinism. Set a fixed value for random seed so that when the parameters are initialized, they are initialized same across all experiments.
torch.manual_seed(42)

# If you are using CUDA, also set the seed for it
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Set the seed for NumPy
np.random.seed(42)

Using device: cpu


Here we define **RiverData** a custom Dataset class to load the dataset we have. It extends the Pytorch Dataset class.  
- We need to define \_\_init__() function which can be used for loading data from the file and optionally for data preprocessing.
- Thereafter we define \_\_len__() function which gives the length of dataset.
- Then we define \_\_getitem__() function which returns an instance of (feature, label) tuple which can be used for model training.
  For our time series data, feature means the past values to be used for training and label means the future values to be predicted.

In [ ]:
class RiverData(torch.utils.data.Dataset):

    def __init__(self, df, target, datecol, seq_len, pred_len):
        self.df = df
        self.datecol = datecol
        self.target = target
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.setIndex()


    def setIndex(self):
        self.df.set_index(self.datecol, inplace=True)


    def __len__(self):
        return len(self.df) - self.seq_len - self.pred_len


    def __getitem__(self, idx):
        if len(self.df) <= (idx + self.seq_len+self.pred_len):
            raise IndexError(f"Index {idx} is out of bounds for dataset of size {len(self.df)}")
        df_piece = self.df[idx:idx+self.seq_len].values
        feature = torch.tensor(df_piece, dtype=torch.float32)
        label_piece = self.df[self.target][idx + self.seq_len:  idx+self.seq_len+self.pred_len].values
        label = torch.tensor(label_piece, dtype=torch.float32)
        return (feature, label)

### Normalize the data

In [ ]:
df = pd.read_csv(path)
raw_df = df.drop('DATE', axis=1, inplace=False)
scaler = MinMaxScaler()

# Apply the transformations
df_scaled = scaler.fit_transform(raw_df)

df_scaled = pd.DataFrame(df_scaled, columns=raw_df.columns)
df_scaled['DATE'] = df['DATE']
df = df_scaled

Some advanced Python syntax has been used here. \
*common_args : it's used to pass arguments to a function, where common_args represents a python list \
**common_args: it's used to pass arguments to a function, where common_args represents a python dictionary

In [ ]:

train_size = int(0.7 * len(df))
test_size = int(0.2 * len(df))
val_size = len(df) - train_size - test_size

seq_len = 13
pred_len = 1
num_features = 7
num_layers = 1


common_args = ['gauge_height', 'DATE', seq_len, pred_len]
train_dataset = RiverData(df[:train_size], *common_args)
val_dataset = RiverData(df[train_size: train_size+val_size], *common_args)
test_dataset = RiverData(df[train_size+val_size : len(df)], *common_args)


In [ ]:
# Important parameters

BATCH_SIZE = 128 # keep as big as can be handled by GPU and memory
SHUFFLE = False # we don't shuffle the time series data
DATA_LOAD_WORKERS = 1 # it depends on amount of data you need to load
learning_rate = 1e-3

In [ ]:
from torch.utils.data import DataLoader

common_args = {'batch_size': BATCH_SIZE, 'shuffle': SHUFFLE}
train_loader = DataLoader(train_dataset, **common_args)
val_loader = DataLoader(val_dataset, **common_args)
test_loader = DataLoader(test_dataset, **common_args)

### Here we define our PyTorch model.

BasicTransformerNetwork is the model class, it extends the Module class provided by Pytorch. \
- We define \_\_init__() function. It sets up layers and defines the model parameters.
- Also, we define forward() function which defines how the forwared pass computation occurs
- We also implement PositionalEncoding class which is an important part of transformer

In [ ]:
# The transformer implementation in pytorch doesn't implement the
# positional encoding which is an essential part of the transforemer model

# Provide more description of positional encoding
class PositionalEncoding(torch.nn.Module):
    def __init__(self, d_model, pos_enc_dropout, max_len=5000):
        super().__init__();
        self.dropout = torch.nn.Dropout(p=pos_enc_dropout)

        Xp = torch.zeros(max_len, d_model) # max_len x d_model
        position = torch.arange(0, max_len).unsqueeze(1) # max_len x 1

        # Generates an exponentially decreasing series of numbers
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)) #length: d_model/2

        #Applying sine to even indices in the array; 2i
        Xp[:, 0::2] = torch.sin(position.float() * div_term)

        #Applying cosine to odd indices in the array; 2i + 1
        Xp[:, 1::2] = torch.cos(position.float() * div_term)

        Xp = Xp.unsqueeze(1)
        self.register_buffer('Xp', Xp)

    def forward(self, x):
        x  = x + self.Xp[:x.size(0)]
        return self.dropout(x)



class BasicTransformerNetwork(torch.nn.Module):

    def __init__(self, seq_len, pred_len, enc_layer_dropout, pos_enc_dropout):
        # call the constructor of the base class
        super().__init__()
        self.model_type = 'Transformer'
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.num_features = num_features

        # I don't think the embedding size should be this big. We will see.
        self.embedding_size = 128 #The features are converted to 128 embeddings
        self.num_layers = num_layers
        self.pos_encoder = PositionalEncoding(self.embedding_size, pos_enc_dropout, 10000)

        # dim_feedforward = 4 * d_model
        # layer_norm_eps: A very small number (epsilon) added to the denominator during the Layer Normalization calculation.
        self.encLayer = torch.nn.TransformerEncoderLayer(d_model=self.embedding_size, nhead=8,
                                                 dim_feedforward=256, dropout=enc_layer_dropout, activation="relu",
                                                 layer_norm_eps=1e-05, batch_first=True)

        self.transformerEnc = torch.nn.TransformerEncoder(self.encLayer, num_layers=self.num_layers)

        self.input_fc = torch.nn.Linear(self.num_features, self.embedding_size)

        self.prediction_head = torch.nn.Linear(self.embedding_size, self.pred_len)

        # Create causal mask
        self.register_buffer('causal_mask', self._generate_causal_mask(seq_len))


    def _generate_causal_mask(self, seq_len):
        """
        Generate causal mask for transformer encoder.
        Returns upper triangular matrix with -inf in upper triangle (excluding diagonal)
        """
        mask = torch.triu(torch.full((seq_len, seq_len), float('-inf')), diagonal=1)
        return mask


    def forward(self, x):
        x = self.input_fc(x) * np.sqrt(self.embedding_size)
        x = self.pos_encoder(x)
        out = self.transformerEnc(x, mask=self.causal_mask)
        last_embedding = out[:, -1, :]
        prediction = self.prediction_head(last_embedding)
        prediction = prediction.squeeze(-1)
        return prediction
# Note that the gradients are stored inside the FC layer objects
# For each training example we need to get rid of these gradients

In [ ]:
print(torch.__version__)

2.2.1+cu121


In [ ]:
loss = torch.nn.MSELoss()


In [ ]:
for i, (f,l) in enumerate(train_loader):
    print('features shape: ', f.shape)
    print('labels shape: ', l.shape)
    break

features shape:  torch.Size([512, 13, 7])
labels shape:  torch.Size([512, 1])


In [ ]:
# define metrics
import numpy as np
epsilon = np.finfo(float).eps

def wape_function(y, y_pred):
    """Weighted Average Percentage Error metric in the interval [0; 100]"""
    y = np.array(y)
    y_pred = np.array(y_pred)
    nominator = np.sum(np.abs(np.subtract(y, y_pred)))
    denominator = np.add(np.sum(np.abs(y)), epsilon)
    wape = np.divide(nominator, denominator) * 100.0
    return wape

def nse_function(y, y_pred):
    y = np.array(y)
    y_pred = np.array(y_pred)
    return (1-(np.sum((y_pred-y)**2)/np.sum((y-np.mean(y))**2)))


def evaluate_model(model, data_loader):
    # following line prepares the model for evaulation mode. It disables dropout and batch normalization if they have
    # are part of the model. For our simple model it's not necessary. Still I'm going to use it.

    model.eval()
    all_outputs = torch.empty(0, pred_len)
    all_labels = torch.empty(0, pred_len)
    for inputs, labels in data_loader:
        inputs = inputs.to(device)
        with torch.no_grad():
            outputs = model(inputs).detach().cpu().unsqueeze(1)
        all_outputs = torch.vstack((all_outputs, outputs))
        all_labels = torch.vstack((all_labels, labels))

    avg_val_loss = loss(all_outputs, all_labels)
    nse = nse_function(all_labels.numpy(), all_outputs.numpy())
    wape = wape_function(all_labels.numpy(), all_outputs.numpy())

    print(f'NSE : {nse}', end=' ')
    print(f'WAPE : {wape}', end=' ')
    print(f'Validation Loss: {avg_val_loss}')
    model.train()
    return avg_val_loss


In [ ]:
from optuna.samplers import TPESampler
def objective(trial):
    # Here we define the search space of the hyper-parameters. Optuna uses byaesian optimization to find the optimal values of the hyperparameters.
    learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)
    pos_enc_dropout = trial.suggest_uniform('pos_enc_dropout', 0.05, 0.3)
    enc_layer_dropout = trial.suggest_uniform('enc_layer_dropout', 0.1, 0.5)


    model = BasicTransformerNetwork(seq_len, pred_len, pos_enc_dropout, enc_layer_dropout)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)

    num_epochs = 10
    best_val_loss = float('inf')
    patience = 1

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = []
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs).unsqueeze(1)
            loss_val = loss(outputs, labels)

            # calculate gradients for back propagation
            loss_val.backward()

            # update the weights based on the gradients
            optimizer.step()

            # reset the gradients, avoid gradient accumulation
            optimizer.zero_grad()
            epoch_loss.append(loss_val.item())

        avg_train_loss = sum(epoch_loss)/len(epoch_loss)
        print(f'Epoch {epoch+1}: Traning Loss: {avg_train_loss}', end=' ')
        avg_val_loss = evaluate_model(model, val_loader)

        # Check for improvement
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # Save the best model
            torch.save(model.state_dict(), 'best_model_trial.pth')
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print('Early stopping!')
                # Load the best model before stopping
                model.load_state_dict(torch.load('best_model_trial.pth'))
                break

        # Report intermediate objective value
        trial.report(best_val_loss, epoch)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_loss

# Default sampler is TPESampler (Tree-structured Parzen Estimator).
# This sampler is based on independent sampling and uses a Bayesian optimization approach to efficiently explore
# the hyperparameter search space by building probability models of objective values.

study = optuna.create_study(direction='minimize', sampler=TPESampler())

# normally you run 100s of trials.
study.optimize(objective, n_trials=20)

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('  Value (Best Validation Loss):', trial.value)
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')


[I 2024-11-19 17:19:04,438] A new study created in memory with name: no-name-2213175a-c3b5-4b56-bf90-164e77b99d0f
/local_scratch/slurm.1125506/ipykernel_3266291/2234128446.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
/local_scratch/slurm.1125506/ipykernel_3266291/2234128446.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)
/local_scratch/slurm.1125506/ipykernel_3266291/2234128446.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v

Epoch 1: Traning Loss: 0.02526580304560975 NSE : -2.6605618000030518 WAPE : 117.91830153052135 Validation Loss: 0.06249501183629036
Epoch 2: Traning Loss: 0.01687834267906353 NSE : -0.08283412456512451 WAPE : 59.316866371845336 Validation Loss: 0.018486706539988518
Epoch 3: Traning Loss: 0.017058147678889222 

[I 2024-11-19 17:19:30,536] Trial 0 finished with value: 0.018486706539988518 and parameters: {'lr': 0.009710522346682626, 'weight_decay': 8.829487000121852e-05, 'pos_enc_dropout': 0.15459847164742846, 'enc_layer_dropout': 0.41149718189368745}. Best is trial 0 with value: 0.018486706539988518.


NSE : -0.22743380069732666 WAPE : 65.16451114665495 Validation Loss: 0.020955391228199005
Early stopping!


/local_scratch/slurm.1125506/ipykernel_3266291/2234128446.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
/local_scratch/slurm.1125506/ipykernel_3266291/2234128446.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)
/local_scratch/slurm.1125506/ipykernel_3266291/2234128446.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  pos_enc_dropout = trial.suggest_uniform('pos_enc_dropout', 0.05, 0.3)
/local_s

...

NSE : 0.11953943967819214 WAPE : 55.82810916418224 Validation Loss: 0.015031680464744568
Epoch 1: Traning Loss: 0.024789026397077113 NSE : 0.05429583787918091 WAPE : 54.14845162458103 Validation Loss: 0.016145553439855576
Epoch 2: Traning Loss: 0.018053230623889544 

[I 2024-11-19 17:26:48,337] Trial 18 finished with value: 0.016145553439855576 and parameters: {'lr': 0.00022163253578322888, 'weight_decay': 0.0004269280238541515, 'pos_enc_dropout': 0.18892590096062278, 'enc_layer_dropout': 0.33171824077582357}. Best is trial 7 with value: 0.0038622659631073475.


NSE : -0.039455533027648926 WAPE : 49.55015713004888 Validation Loss: 0.017746126279234886
Early stopping!
Epoch 1: Traning Loss: 0.018440998612600498 NSE : -0.2747478485107422 WAPE : 51.922105120073134 Validation Loss: 0.021763157099485397
Epoch 2: Traning Loss: 0.01293874411905175 

[I 2024-11-19 17:27:04,721] Trial 19 finished with value: 0.021763157099485397 and parameters: {'lr': 0.0007367874301143976, 'weight_decay': 5.759965462648816e-05, 'pos_enc_dropout': 0.1237335792328222, 'enc_layer_dropout': 0.21557865621541583}. Best is trial 7 with value: 0.0038622659631073475.


NSE : -0.6221939325332642 WAPE : 78.81908422269223 Validation Loss: 0.02769494242966175
Early stopping!
Number of finished trials: 20
Best trial:
  Value (Best Validation Loss): 0.0038622659631073475
  Params:
    lr: 0.0005950865555327729
    weight_decay: 0.0001651786938362955
    pos_enc_dropout: 0.11007064752567254
    enc_layer_dropout: 0.1947265400415874


In [ ]:
# Plot the results with the metrics inside it

In [ ]:
import optuna.visualization as vis

# Optimization history
fig1 = vis.plot_optimization_history(study)
fig1.write_html("optimization_history_transformer.html")